In [1]:
import pandas as pd
import numpy as np
import os 

In [489]:
cwd = os.getcwd()

df = pd.read_excel(cwd+"/B2B Data Provider-sample1.xlsx", index_col=0)

#replace all Nan values with -
df = df.replace(np.nan, "-")

In [490]:
print(df.columns)

Index(['Starting Price - Monthly', 'Starting Price - Annually',
       'Pricing Range', 'Pricing comments', 'Additional Pricing Info',
       'Website', 'Contract', 'Solutions', 'Unlimited Data Plan',
       'CRM Integrations', 'SEPI', 'SalesTeamType', 'Free Trial',
       'ContactEnrichment', 'IntentData', 'Other', 'location'],
      dtype='object')


In [491]:
#removing unnecessary columns
df_soft = df.drop(['Additional Pricing Info','Website','Unlimited Data Plan', 'Pricing Range', 'Pricing comments', 'SalesTeamType','Free Trial','Other'], axis=1)

#drop the pricing from data and this can be used to create a feature matrix
df_pro = df_soft.drop(['Starting Price - Monthly', 'Starting Price - Annually'], axis=1)
df_pro

,Contract,Solutions,CRM Integrations,SEPI,ContactEnrichment,IntentData,location
Company,,,,,,,
ZoomInfo,Annual,"email, workphone, mobile","Microsoft Dynamics,HubSpot,Salesforce,Pipedriv...","Outreach,Salesloft",Yes,Yes,"Domestic,International"
Cognism,Annual,"email, workphone, mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamics",Salesloft,Yes,Yes,International
Uplead,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot,Insightly,Pipedrive,Zoho,Mi...","Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close",Yes,Yes,"Domestic,International"
Lead411,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot,Pipedrive,Microsoft Dynamic...",-,Yes,Yes,"Domestic,International"
Apollo,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot","Salesloft,Outreach",Yes,No,"Domestic,International"
Seamless,Annual,"email, workphone, mobile","Salesforce,HubSpot,Microsoft Dynamics,Keap,Zoh...","Salesloft,Outreach",Yes,No,"Domestic,International"
RocketReach,Annual or Monthly,"email, workphone, mobile","Salesforce,HubSpot","Salesloft,Outreach",No,No,"Domestic,International"
Lusha,Annual or Monthly,"email, workphone","Salesforce,HubSpot","Salesloft,Outreach",Yes,No,"Domestic,International"
Crunchbase,Annual or Monthly,email,Salesforce,-,No,No,"Domestic,International"


In [492]:
#convert all the contract to list values
df_pro['Contract'] = df_pro['Contract'].str.replace(" or ", ",").str.split(',')
#convert all solutions values to list 
df_pro['Solutions'] = df_pro['Solutions'].str.split(",")
#convert all the CRM values to list
df_pro['CRM Integrations'] = df_pro['CRM Integrations'].str.split(",")
#convert all SEPI values to list
df_pro['SEPI'] = df_pro['SEPI'].str.split(",")
#convert all location values to list
df_pro['location'] = df_pro['location'].str.split(",")

In [493]:
#set the contract first
for index,row in df_pro.iterrows():
    for contract in row['Contract']:
        df_pro.at[index,contract] =1
    

In [494]:
#set the solutions 
for index,row in df_pro.iterrows():
    for sol in row['Solutions']:
        df_pro.at[index,sol] =1
    

In [495]:
#set the CRM
for index,row in df_pro.iterrows():
    for crm in row['CRM Integrations']:
        df_pro.at[index,crm] = 1

In [496]:
#set the Sales engagement matrix
for index,row in df_pro.iterrows():
    for sepi in row['SEPI']:
        df_pro.at[index,sepi] = 1

In [497]:
#set the location matric
for index,row in df_pro.iterrows():
    for loc in row['location']:
        df_pro.at[index,loc] = 1

In [498]:
#replace the Yes and no values to 1 and 0 respectively for ContactEnrichment and IntentData columns
df_pro = df_pro.replace('Yes', 1.0)
df_pro = df_pro.replace('No', 0)


In [499]:
#now drop all the initial columns that is not a matrix
df_pro =df_pro.drop(['Contract','Solutions','CRM Integrations','SEPI','location', '-'], axis=1)

In [500]:
#lastly replace all the nan values with 0
df_pro= df_pro.replace(np.nan, 0)

In [501]:
#check the shape
df_pro.shape

(10, 32)

In [502]:
#treated dataframe 
df_pro

,ContactEnrichment,IntentData,Annual,Monthly,email,workphone,mobile,Microsoft Dynamics,HubSpot,Salesforce,...,Freshsales,Outreach,Salesloft,Reply,Woodpecker,Lemlist,Close,Freshworks,Domestic,International
Company,,,,,,,,,,,,,,,,,,,,,
ZoomInfo,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Cognism,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Uplead,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
Lead411,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Apollo,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Seamless,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
RocketReach,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Lusha,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Crunchbase,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


**Now the idea is to cross multiply the values user values with and find the weighted values**

## USER data

In [637]:
df_user = pd.read_csv("user_data.csv", index_col=0)
df_user

,data,contract,duration,price,location,user,tools,features,crm
0,"['Email', 'Work Phone', 'Mobile']","['Monthly', 'Annual']",['Monthly'],800,['All'],2,['All'],"['ContractEnrichment', 'IntentData']",['All']


In [638]:
df_user['data'] = df_user['data'].item().replace("'", "").strip("[]").lower()
df_user['data'] = df_user['data'].str.split(",")

In [639]:
df_user['contract'] = df_user['contract'].item().replace("'", "").strip("[]")
df_user['contract'] = df_user['contract'].str.split(",")

In [640]:
df_user['duration'] = df_user['duration'].item().replace("'", "").strip("[]")

In [641]:
df_user['price'] = df_user['price'].item()

In [642]:
if df_user['location'].item().replace("'", "").strip("[]") == "All":
    df_user['location'] = [['International', 'Domestic']]


In [643]:
if df_user['tools'].item().replace("'", "").strip("[]").replace(" ","") == "All":
    df_user['tools']= [['Outreach', 'Salesloft','Reply','Woodpecker','Lemlist','Close','HubSpot','Salesforce','Pipedrive','Nutshell','Zoho','Freshworks']]
else:
    df_user['tools'] = df_user['tools'].item().replace("'", "").strip("[]").split(",")

In [644]:
df_user['features'] = df_user['features'].item().replace("'", "").strip("[]").replace(" ","")
df_user['features'] = df_user['features'].str.split(",")

In [645]:
if df_user['crm'].item().replace("'", "").strip("[]").replace(" ","") == "All":
    df_user['crm' ]= [['Microsoft Dynamics','HubSpot','Salesforce','Pipedrive','SugarCRM','Zoho','Insightly','Copper','Nimble','Infusionsoft','Capsule','Nutshell','Base','Vtiger','Keap','Freshsales']]
else:
    df_user['crm'] = df_user['crm'].item().replace("'", "").strip("[]").split(",")

In [646]:
#we will remove the price for now 
df_user_matrix = df_user.drop(['price','user'], axis=1)

In [647]:
df_user_matrix

,data,contract,duration,location,tools,features,crm
0,"[email, work phone, mobile]","[Monthly, Annual]",Monthly,"[International, Domestic]","[Outreach, Salesloft, Reply, Woodpecker, Lemli...","[ContractEnrichment, IntentData]","[Microsoft Dynamics, HubSpot, Salesforce, Pipe..."


In [648]:
#set the data first
for index,row in df_user_matrix.iterrows():
    for data in row['data']:
        df_user_matrix.at[index,data] =1

In [649]:
#set the contract first
for index,row in df_user_matrix.iterrows():
    for contract in row['contract']:
        df_user_matrix.at[index,contract] =1

In [650]:
#set the location first
for index,row in df_user_matrix.iterrows():
    for loc in row['location']:
        df_user_matrix.at[index,loc] =1

In [651]:
#set the tools first
for index,row in df_user_matrix.iterrows():
    for tool in row['tools']:
        df_user_matrix.at[index,tool] =1

In [652]:
#set the features first
for index,row in df_user_matrix.iterrows():
    for feat in row['features']:
        df_user_matrix.at[index,feat] =1

In [653]:
#set the crm first
for index,row in df_user_matrix.iterrows():
    for crm in row['crm']:
        df_user_matrix.at[index,crm] =1

In [654]:
#rename column Work phone
df_user_matrix = df_user_matrix.rename(columns={' work phone':'workphone'})


In [655]:
df_user_matrix = df_user_matrix.iloc[:,7:]


In [656]:
df_user_matrix

,email,workphone,mobile,Monthly,Annual,International,Domestic,Outreach,Salesloft,Reply,...,SugarCRM,Insightly,Copper,Nimble,Infusionsoft,Capsule,Base,Vtiger,Keap,Freshsales
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [669]:
for i,item in enumerate(df_user_matrix):
    item = item.lstrip(" ")
    if item in df_pro.columns:
        df_pro[item] = df_pro[item]+1
        print(i,item)

0 email
3 Monthly
4 Annual
5 International
6 Domestic
7 Outreach
8 Salesloft
9 Reply
10 Woodpecker
11 Lemlist
12 Close
13 HubSpot
14 Salesforce
15 Pipedrive
16 Nutshell
17 Zoho
18 Freshworks
20 IntentData
21 Microsoft Dynamics
22 SugarCRM
23 Insightly
24 Copper
25 Nimble
26 Infusionsoft
27 Capsule
28 Base
29 Vtiger
30 Keap
31 Freshsales


In [675]:
df_pro.sum(axis=1).sort_values(ascending=False)

Company
Uplead         52.0
Lead411        49.0
Seamless       47.0
ZoomInfo       45.0
Cognism        41.0
Apollo         41.0
RocketReach    40.0
Lusha          40.0
Snov.io        39.0
Crunchbase     35.0
dtype: float64